In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# BigQuery Geospatial Visualization in Colab


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/bigquery-utils/blob/master/notebooks/bigquery_geospatial_visualization.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fbigquery-utils%2Fmaster%2Fnotebooks%2Fbigquery_geospatial_visualization.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/GoogleCloudPlatform/bigquery-utils/blob/master/notebooks/bigquery_geospatial_visualization.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/bigquery-utils/blob/master/notebooks/bigquery_geospatial_visualization.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>



| | |
|-|-|
| Author(s) |  [Bijan Vakili](https://github.com/bijanvakili/) and [Alicia Williams](https://github.com/aliciawilliams/)

# Introduction
---

The goal of this notebook is to walk through the steps for visualizing geospatial data in a [Colab](https://colab.research.google.com/) notebook. It will source geospatial data from [Google BigQuery](https://cloud.google.com/bigquery).

We will go through:
* Authenticating to Google Cloud
* Reading data from BigQuery into Colab
* Using Python data science tools to perform transformations and analysis
* Using `geemap` and `pydeck` to render scatter plots, polygons, choropleths, and heatmaps



You can also watch the YouTube video which walks through this notebook: [Visualizing BigQuery geospatial data in Colab](https://www.youtube.com/watch?v=t_q-qLa1lX0).

<a href="https://www.youtube.com/watch?v=t_q-qLa1lX0" target="_blank">
  <img src="https://img.youtube.com/vi/t_q-qLa1lX0/maxresdefault.jpg" alt="Visualizing BigQuery geospatial data in Colab" width="500">
</a>

## Datasets and Analysis Goals

The datasets we'll be working with include:

* [San Francisco Ford GoBike Share](https://console.cloud.google.com/bigquery(cameo:product/san-francisco-public-data/sf-bike-share)
* [San Francisco Neighborhoods](https://console.cloud.google.com/bigquery?ws=!1m4!1m3!3m2!1sbigquery-public-data!2ssan_francisco_neighborhoods)
* [San Francisco Police Department (SFPD) Reports](https://console.cloud.google.com/bigquery(cameo:product/san-francisco-public-data/sfpd-reports))

These [public datasets](https://cloud.google.com/bigquery/public-data) are hosted in Google BigQuery and is included in BigQuery's 1TB/mo of free tier processing. This means that each user receives 1TB of free BigQuery processing every month, which can be used to run queries on this public dataset. Read this [documentation page](https://cloud.google.com/bigquery/docs/sandbox) to learn how to get started quickly using BigQuery to access public datasets.

Our goals are to query and plot the following information in San Francisco:
* All bike share stations using `ScatterplotLayer`
* All neighborhoods polygons using a `GeoJSONLayer`
* Choropleth of bike share station density (stations per square kilometer) per neighborhood using a `PolygonLayer`
* Continous density of SFPD incidents using a `HeatmapLayer`

## Related Data Science Tools/Techniques

In each step of the above journey, we show how BigQuery and Python data science and plotting libraries (via Colab) can work together to enable this type of analysis at scale. Some key tools and techniques we'll employ:
* [colabtools](https://github.com/googlecolab/colabtools) (`google.colab` python modules)
* [Google BigQuery](https://cloud.google.com/bigquery/what-is-bigquery)
* Python Data Science Libraries: [pandas](https://pandas.pydata.org/)
* Python Geospatial Libraries:
    * [geopandas](https://geopandas.org/en/stable/index.html) to extend the datatypes used by `pandas` to allow spatial operations on geometric types
    * [shapely](https://shapely.readthedocs.io/en/stable/index.html) for manipulation and analysis of individual planar geometric objects    
* [branca](https://python-visualization.github.io/branca/) to generate generate HTML + JS colormaps
* [pydeck](https://deckgl.readthedocs.io/en/latest/) for chart visualization which is powered by [deck.gl](https://deck.gl/#/)
* [geemap](https://geemap.org/) for visualization with `pydeck` and `earthengine-api`
    * [geemap.deck](https://geemap.org/deck/) module
    * [github source](https://github.com/gee-community/geemap)
* [h3](https://uber.github.io/h3-py/intro.html) for Hierarchical Geospatial Indexing System


## BigQuery Pricing and Cost of Running This Notebook
If you don’t already have a Google Cloud project, there are 2 no-cost options available for accessing this data:

1. Sign up for [BigQuery sandbox](https://cloud.google.com/bigquery/docs/sandbox) to try it without enabling billing.
2. If you want to experiment with multiple Google Cloud products, activate the [free trial](https://cloud.google.com/free/) ($300 credit for up to 90 days).

You can use [BigQuery's free tier](https://cloud.google.com/bigquery/pricing#free-tier) to store and analyze a certain amount of data at no cost (1 TB query, 10 GB storage capacity per month), even after a free trial period.

Running this notebook in its entirety, with the default values, should fall within the free usage tier and shouldn't cost you anything. However, changing the default settings and/or running it multiple times may incur additional charges. See the [BigQuery pricing guide](https://cloud.google.com/bigquery/pricing) for more information.

This is not an official Google product but sample code provided for an educational purpose.

## Help Google improve Tutorials

* File issues [here](https://github.com/GoogleCloudPlatform/bigquery-utils/issues) on Github.
* Submit feedback [here](https://docs.google.com/forms/d/e/1FAIpQLSdNSDbBM2rohqtqWtIPgKD_14oLc8TPqqdtKD11oqsrtA8_NQ/viewform) via a Google Form.



#Setup

## GCP Authentication

We set up our Colab by installing, importing, and enabling the usage of a few Python libraries within Colab, as well as authenticating this Colab runtime with the appropriate Google Cloud `GCP_PROJECT_ID`. This follows closely the instructions in the ["Getting started with BigQuery"](https://colab.sandbox.google.com/notebooks/bigquery.ipynb#scrollTo=SeTJb51SKs_W)
example Colab.

The authentication step in the next cell will require manually going through some pop-up screens and copy/pasting an authentication code from another window back into the cell to complete (on the 1st run; may run automatically thereafter).

In [ ]:
# REQUIRED: authenticate with GCP project
GCP_PROJECT_ID = ""  #@param {type:"string"}

from google.colab import auth
from google.colab import userdata

auth.authenticate_user(project_id=GCP_PROJECT_ID)

# Set GMP_API_KEY to none
GMP_API_KEY = None

## (Optional) GMP Authentication

If you wish to use Google Maps Platform (GMP) as the map provider for base maps, you must provide a [Google Maps Platform API key](https://developers.google.com/maps/documentation/javascript/get-api-key).  This notebook can retrieve it from your [Colab Secrets](https://colab.sandbox.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb#scrollTo=dEoigYI9Jw_K) if you provide a key name.

If no key name is provided, `pydeck` will default to using the `carto` map provided.

In [ ]:
GMP_API_SECRET_KEY_NAME = "" #@param {type:"string"}

if GMP_API_SECRET_KEY_NAME:
  GMP_API_KEY = userdata.get(GMP_API_SECRET_KEY_NAME) if GMP_API_SECRET_KEY_NAME else None
else:
  GMP_API_KEY = None

##Enable BigQuery API

In [ ]:
!gcloud services enable \
    --project $GCP_PROJECT_ID \
    bigquery.googleapis.com

##(Optional) Enable Google Maps Javascript API

In [ ]:
!gcloud services enable \
    --project $GCP_PROJECT_ID \
    "maps-backend.googleapis.com"

# Install Packages

In [ ]:
!pip install pydeck>=0.9 h3>=4.2

##Import libraries

In [ ]:
import branca
import geemap.deck as gmdk
import h3
import pydeck as pdk
import geopandas as gpd
import shapely

Enable interactive tables for `pandas` DataFrames in Colab.

> See [this article](https://colab.google/articles/alive) for more details.

In [ ]:
# Enable displaying pandas data frames as interactive tables by default
from google.colab import data_table
data_table.enable_dataframe_formatter()

# Shared Routines

We create `display_pydeck_map` here to render charts using `pydeck`.  This
involves creating a `pydeck.Map` instance and then adding layers (`pydeck.Layer`) to the map instance. Note that while the specific examples in this notebook only use one layer at a time, this routine is written to handle multiple layers for `pydeck`.

In [ ]:
MAP_PROVIDER_GOOGLE = pdk.bindings.base_map_provider.BaseMapProvider.GOOGLE_MAPS.value

# Shared routine for rendering layers on a map using geemap.deck
def display_pydeck_map(layers, view_state, **kwargs):
  deck_kwargs = kwargs.copy()

  # use Google Maps as the base map only if the API key is provided
  if GMP_API_KEY:
    deck_kwargs.update({
      "map_provider": MAP_PROVIDER_GOOGLE,
      "map_style": pdk.bindings.map_styles.GOOGLE_ROAD,
      "api_keys": {MAP_PROVIDER_GOOGLE: GMP_API_KEY},
    })

  m = gmdk.Map(initial_view_state=view_state, ee_initialize=False, **deck_kwargs)

  for layer in layers:
    m.add_layer(layer)
  return m

# Create a scatter plot with `ScatterplotLayer`

Scatter plots are most useful when you need to review a subsample of individual points, such as when validating data. This is sometimes referred to as "spot checking".

In this section, we'll create a scatter plot of all bike share stations listed in the [San Francisco Ford GoBike Share](https://console.cloud.google.com/bigquery(cameo:product/san-francisco-public-data/sf-bike-share) public dataset.

## Import dataset from BigQuery: Ford GoBike stations in San Francisco

We use the `%%bigquery` magic to run a query and add the [`--use_geodataframe` parameter](https://googleapis.dev/python/bigquery-magics/latest/magics.html#module-bigquery_magics.bigquery:~:text=%2D%2Duse_geodataframe%20%3Cparams%3E) followed by the geometry column name in order to return the results in a `geopandas.GeoDataFrame`.

In [ ]:
# NOTE: For the purposes of this tutorial, we ignore the denormalized 'lat' and 'lon' columns which are decomposed components of the geometry
%%bigquery gdf_sf_bikestations --project {GCP_PROJECT_ID} --use_geodataframe station_geom

SELECT
  station_id,
  name,
  short_name,
  station_geom
FROM
  `bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info`

## Rendering a Scatter plot

This example demonstrates how to use [pydeck.Layer](https://deckgl.readthedocs.io/en/latest/layer.html#pydeck.bindings.layer.Layer) and the [ScatterPlotLayer](https://deck.gl/docs/api-reference/layers/scatterplot-layer) to render individual points as circles.  This requires extracting the longitude and latitude as x and y coordinates respectively from the `station_geom` column.

In [ ]:
gdf_sf_bikestations.info()

In [ ]:
gdf_sf_bikestations.head()

Since `gdf_sf_bikestations` is a `geopandas.GeoDataFrame`, coordinates can be accessed directly from its `station_geom` geometry column. The code retrieves the longitude using the column's `.x` attribute and the latitude using its `.y` attribute, storing them in new `longitude` and `latitude` columns.

In [ ]:
# Extract the longitude (x) and latitude (y)
gdf_sf_bikestations["longitude"] = gdf_sf_bikestations["station_geom"].x
gdf_sf_bikestations["latitude"] = gdf_sf_bikestations["station_geom"].y

The cell below renders a scatter plot.

If you wish to customize it, please review the [pydeck.Layer](https://deckgl.readthedocs.io/en/latest/layer.html#pydeck.bindings.layer.Layer) API docs and the [ScatterPlotLayer](https://deck.gl/docs/api-reference/layers/scatterplot-layer) docs to discover other parameters.

In [ ]:
# render a scatter plot using pydeck with the extracted longitude and latitude columns
# in the gdf_sf_bikestations geopandas.GeoDataFrame.
scatterplot_layer = pdk.Layer(
  "ScatterplotLayer",
  id="bike_stations_scatterplot",
  data=gdf_sf_bikestations,
  get_position=['longitude', 'latitude'],
  get_radius=100,
  get_fill_color=[255, 0, 0, 140],  # Adjust color as desired
  pickable=True,
)

view_state = pdk.ViewState(latitude=37.77613, longitude=-122.42284, zoom=12)
display_pydeck_map([scatterplot_layer], view_state)

# Visualize polygons with a `GeoJSONLayer`

BigQuery's [GEOGRAPHY](https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types#geography_type) data type can represent a geometry value or geometry collection.  Examples of shapes include:
* points
* lines
* polygons
* multi-polygons


Sometimes you are provided geospatial data without knowing in advance its expected shapes.  

In this case, visualizing the data can help you discover the shapes enabling further analysis. This visualization can be done by converting the geographic data to GeoJSON and then visualizing it with a `GeoJSONLayer`.

As an example, we'll import geographic data from the [San Francisco Neighborhoods](https://console.cloud.google.com/bigquery?ws=!1m4!1m3!3m2!1sbigquery-public-data!2ssan_francisco_neighborhoods) dataset and then visualize it.

## Import dataset from BigQuery: San Francisco Neighborhoods

In [ ]:
%%bigquery gdf_sanfrancisco_neighborhoods --project {GCP_PROJECT_ID} --use_geodataframe geometry

SELECT
  neighborhood,
  neighborhood_geom AS geometry,
  ST_AREA(neighborhood_geom) AS area_sq_meters
FROM
  `bigquery-public-data.san_francisco_neighborhoods.boundaries`

In [ ]:
gdf_sanfrancisco_neighborhoods.info()

In [ ]:
# peek at the first item
gdf_sanfrancisco_neighborhoods.head(1)

We see above that the data is a `POLYGON`.

The cell below renders the data by relying on `pydeck` to convert each `shapely` object instance in the `geometry` column to the [GeoJSON format](https://datatracker.ietf.org/doc/html/rfc7946).

If you wish to customize the visualization, please review the [pydeck.Layer](https://deckgl.readthedocs.io/en/latest/layer.html#pydeck.bindings.layer.Layer) API docs and the [GeoJsonLayer](https://deck.gl/docs/api-reference/layers/geojson-layer) docs to discover other parameters.

In [ ]:
geojson_layer = pdk.Layer(
    'GeoJsonLayer',
    id="sf_neighborhoods",
    data=gdf_sanfrancisco_neighborhoods,
    get_line_color=[127, 0, 127, 255],
    get_fill_color=[60, 60, 60, 50],
    get_line_width=100,
    pickable=True,
    stroked=True,
    filled=True,
  )
view_state = pdk.ViewState(latitude=37.77613, longitude=-122.42284, zoom=12)
display_pydeck_map([geojson_layer], view_state)

#Render a choropleth using `PolygonLayer`

If you are exploring data that are polgyons which are not easily converted to GeoJSON, you can consider using a [PolygonLayer](https://deck.gl/docs/api-reference/layers/polygon-layer) instead.  The `PolygonLayer` can process input data of [specific types](https://deck.gl/docs/api-reference/layers/polygon-layer#getpolygon) such as an array of points.

We'll show how to use a `PolygonLayer` to render an array of points and take it a step further to produce a choropleth of bike station density by neighborhood.

In [ ]:
# convert neighborhood area to square kilometers and perform the join and aggregation
gdf_stations_x_neighborhood = gdf_sanfrancisco_neighborhoods.copy()
gdf_stations_x_neighborhood['area_sq_km'] = (
    gdf_stations_x_neighborhood['area_sq_meters'] / 1000000.0
)

# aggregate and count the number of stations per neighborhood
gdf_count_stations = gdf_sanfrancisco_neighborhoods.sjoin(
    gdf_sf_bikestations, how='left', predicate='contains'
)
gdf_count_stations = (
    gdf_count_stations.groupby(by='neighborhood')['station_id']
    .count()
    .rename('num_stations')
)
gdf_stations_x_neighborhood = gdf_stations_x_neighborhood.join(
    gdf_count_stations, on='neighborhood', how='inner'
)

# calculate the normalized density: stations per square kilometer
gdf_stations_x_neighborhood['stations_per_sq_km'] = (
    gdf_stations_x_neighborhood['num_stations'] / gdf_stations_x_neighborhood['area_sq_km']
)
# to simulate non-GeoJSON input data, create 'polygon' column which contains an array of points using the pandas.Series.map method.
gdf_stations_x_neighborhood['polygon'] = gdf_stations_x_neighborhood['geometry'].map(lambda g: list(g.exterior.coords))

In [ ]:
# add a 'fill_color' column for each of the polygons.  this is done by first creating a color map gradient using the branch library.
colormap = branca.colormap.LinearColormap(
  colors=["lightblue", "darkred"],
  vmin=0,
  vmax=gdf_stations_x_neighborhood['stations_per_sq_km'].max(),
)
gdf_stations_x_neighborhood['fill_color'] = gdf_stations_x_neighborhood['stations_per_sq_km'] \
  .map(lambda c: list(colormap.rgba_bytes_tuple(c)[:3]) + [int(0.7 * 255)])   # force opacity of 0.7

The cell below renders a polygon layer.

If you wish to customize it, please review the [pydeck.Layer](https://deckgl.readthedocs.io/en/latest/layer.html#pydeck.bindings.layer.Layer) API docs and the [PolygonLayer](https://www.google.com/url?q=https%3A%2F%2Fdeck.gl%2Fdocs%2Fapi-reference%2Flayers%2Fpolygon-layer) docs to discover other parameters.

In [ ]:
polygon_layer = pdk.Layer(
  'PolygonLayer',
  id="bike_stations_choropleth",
  data=gdf_stations_x_neighborhood,
  get_polygon='polygon',
  get_fill_color='fill_color',
  get_line_color=[0, 0, 0, 255],
  get_line_width=50,
  pickable=True,
  stroked=True,
  filled=True,
)
view_state = pdk.ViewState(latitude=37.77613, longitude=-122.42284, zoom=12)
display_pydeck_map([polygon_layer], view_state)

# Create a heatmap with `HeatmapLayer`

Choropleths are useful when you have meaningful boundaries known in advance. However, when you have lots of data with no known meaningful boundaries, consider using a `HeatmapLayer` to render its continuous density.

For this example, we will use the [San Francisco Police Department (SFPD) Reports](https://console.cloud.google.com/bigquery(cameo:product/san-francisco-public-data/sfpd-reports)) dataset to query and visualize the distribution of incidents in 2015.

For heatmaps, it is recommended to quantize and aggregate the data before rendering.  This example will do so using Carto's [H3](https://docs.carto.com/data-and-analysis/analytics-toolbox-for-bigquery/sql-reference/h3) spatial indexing.

## Import dataset from BigQuery: San Francisco Police Department Incidents

In [ ]:
%%bigquery gdf_incidents --project {GCP_PROJECT_ID} --use_geodataframe location_geography

SELECT
  unique_key,
  location_geography
FROM (
  SELECT
    unique_key,
    SAFE.ST_GEOGFROMTEXT(location) AS location_geography, # WKT string to GEOMETRY
    EXTRACT(YEAR FROM timestamp) AS year,
  FROM `bigquery-public-data.san_francisco_sfpd_incidents.sfpd_incidents` incidents
)
WHERE year = 2015

Quantizing can be done using the [h3](https://github.com/uber/h3-py) python library.  This will aggregate the incident points into hexagons.

Specifically, we use:
* [h3.latlng_to_cell](https://uber.github.io/h3-py/api_verbose.html#h3.latlng_to_cell) to map the incident's position (latitude and longitude) to an H3 cell index. An H3 [resolution](https://h3geo.org/docs/core-library/restable/) of `9`  provides sufficient aggregated hexagons for the heatmap.
* [h3.cell_to_latlng](https://uber.github.io/h3-py/api_verbose.html#h3.cell_to_latlng) to determine the center of each hexagon.

> Please note that you can also use Carto's [H3 functions](https://docs.carto.com/data-and-analysis/analytics-toolbox-for-bigquery/sql-reference/h3) in BigQuery SQL to perform a similar conversion.


In [ ]:
# compute the cell for each incident's latitude and longtitude
H3_RESOLUTION = 9
gdf_incidents['h3_cell'] = gdf_incidents.geometry.apply(
    lambda geom: h3.latlng_to_cell(geom.y, geom.x, H3_RESOLUTION)
)

# aggregate the incidents for each hexagon cell
count_incidents = gdf_incidents.groupby(by='h3_cell')['unique_key'].count().rename('num_incidents')

# construct a new geopandas.GeoDataFrame with the aggregate results.
# add the center of each hexagon for the HeatmapLayer to render
gdf_incidents_x_cell = gpd.GeoDataFrame(data=count_incidents).reset_index()
gdf_incidents_x_cell['h3_center'] = gdf_incidents_x_cell['h3_cell'].apply(h3.cell_to_latlng)
gdf_incidents_x_cell.info()

In [ ]:
gdf_incidents_x_cell.head()

In [ ]:
# convert to a JSON format recognized by the HeatmapLayer
def _make_heatmap_datum(row) -> dict:
  return {
      "latitude": row['h3_center'][0],
      "longitude": row['h3_center'][1],
      "weight": float(row['num_incidents']),
  }

heatmap_data = gdf_incidents_x_cell.apply(_make_heatmap_datum, axis='columns').values.tolist()

The cell below renders a heatmap layer.

If you wish to customize it, please review the [pydeck.Layer](https://deckgl.readthedocs.io/en/latest/layer.html#pydeck.bindings.layer.Layer) API docs and the [HeatmapLayer](https://deck.gl/docs/api-reference/aggregation-layers/heatmap-layer) docs to discover other parameters.

In [ ]:
heatmap_layer = pdk.Layer(
  "HeatmapLayer",
  id="sfpd_heatmap",
  data=heatmap_data,
  get_position=['longitude', 'latitude'],
  get_weight='weight',
  opacity=0.7,
  radius_pixels=99,  # this limitation can introduce artifacts (see above)
  aggregation='MEAN',
)
view_state = pdk.ViewState(latitude=37.77613, longitude=-122.42284, zoom=12)
display_pydeck_map([heatmap_layer], view_state)

# Further Reading

You've learn how to visualize geospatial data stored in BigQuery using `pydeck`.

If you want to learn more about `pydeck` and other `deck.gl` chart types, please see the `pydeck` [Gallery](https://deckgl.readthedocs.io/en/latest/) for examples.  You can also review the `deck.gl` [Layer Catalog](https://deck.gl/docs/api-reference/layers) and [github source](https://github.com/visgl/deck.gl) for other chart types.

If you want to read more about geospatial data, please read the GeoPandas [Getting Started](https://geopandas.org/en/stable/getting_started.html) page and the [User guide](https://geopandas.org/en/stable/docs/user_guide.html).  For geometric object manipulation, please check out the Shapely [user manual](https://shapely.readthedocs.io/en/stable/manual.html).

You can read more in the BigQuery documentation about [geospatial analytics](https://cloud.google.com/bigquery/docs/geospatial-intro) and [geospatial data visualization with BigQuery](https://cloud.google.com/bigquery/docs/geospatial-visualize).

To explore using Earth Engine data with BigQuery for visualization, please see Google Earth Engine [Exporting to BigQuery](https://developers.google.com/earth-engine/guides/exporting_to_bigquery).  For more details on interactive geospatial analysis and visualization with Google Earth Engine, please see [geemap.org](https://geemap.org/).

## Cleaning up

If you created a Google Cloud project specifically for this tutorial, you can [delete the project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects).

If you created a Google Maps Platform API key for this tutorial, it is recommended to [delete the key](https://developers.google.com/maps/api-security-best-practices#deleting-unused-apikeys).

## Help Google improve Tutorials

File issues [here](https://github.com/GoogleCloudPlatform/bigquery-utils/issues) on Github.

Submit feedback [here](https://docs.google.com/forms/d/e/1FAIpQLSdNSDbBM2rohqtqWtIPgKD_14oLc8TPqqdtKD11oqsrtA8_NQ/viewform) via a Google Form.